Importing IMF GDP Per capita data to use as target values\
Data include GDP per capita for every country for all years available.

In [40]:
import pandas as pd
import time
import numpy as np
from datetime import date


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from sklearn.tree import DecisionTreeRegressor

import altair as alt
from vega_datasets import data


In [41]:
imf_target_values_import=pd.read_excel(open('/Users/stephenontko/Documents/University of Michigan/UMSI/UMSI MADS/UMSI MADS 2023/SIADS 699 Capstone/Data/IMF/imf-dm-export-20230215.xlsx','rb'),sheet_name='NGDPDPC')

imf_target_values_import = imf_target_values_import.iloc[1:,:]
print(imf_target_values_import)

    GDP per capita, current prices\n (U.S. dollars per capita)    1980.0  \
1                                          Afghanistan           no data   
2                                              Albania           728.359   
3                                              Algeria          2268.607   
4                                              Andorra           no data   
5                                               Angola           745.119   
..                                                 ...               ...   
227                           Other advanced economies          6886.485   
228                                 Sub-Saharan Africa           no data   
229                                              World          2862.333   
230                                                NaN               NaN   
231                                         ©IMF, 2022               NaN   

       1981.0    1982.0    1983.0    1984.0    1985.0    1986.0     1987.0  \
1     no 

In [42]:
imf_target_values_import.index.name = 'Country'

file_model_df = imf_target_values_import.copy()

file_model_df.index.name = 'Country'

file_model_df['Country Name'] = file_model_df['GDP per capita, current prices\n (U.S. dollars per capita)']
file_model_df.index = file_model_df['Country Name']
file_model_df=file_model_df.drop(columns = ['Country Name','GDP per capita, current prices\n (U.S. dollars per capita)'])

file_model_df.set_axis([int(x) for x in file_model_df.columns],axis = 'columns', inplace=True)

file_model_df=file_model_df.replace('no data',0)
file_model_df=file_model_df.fillna(0)
file_model_df = file_model_df.loc[file_model_df.index.notna()]


country_exclusion_list = [
'Africa (Region)',
'Asia and Pacific',
'Australia and New Zealand',
'Caribbean',
'Central America',
'Central Asia and the Caucasus',
'East Asia',
'Eastern Europe ', 
'Europe',
'Middle East (Region)',
'Myanmar',
'North Africa',
'North America',
'Pacific Islands ', 
'South America',
'South Asia',
'Southeast Asia',
'Sub-Saharan Africa (Region) ',
'Western Europe',
'Western Hemisphere (Region)',
'ASEAN-5',
'Advanced economies',
'Emerging and Developing Asia',
'Emerging and Developing Europe',
'Emerging market and developing economies',
'Euro area',
'European Union',
'Latin America and the Caribbean',
'Major advanced economies (G7)',
'Middle East and Central Asia',
'Other advanced economies',
'Sub-Saharan Africa',
'World',
'nan',
'NaN',
'©IMF, 2022'
]


file_model_df.rename(index={'nan':pd.NA},inplace=True)
file_model_df = file_model_df[~file_model_df.index.isin(country_exclusion_list)].dropna()


# file_model_df.to_csv('/Users/stephenontko/Documents/University of Michigan/UMSI/UMSI MADS/UMSI MADS 2023/SIADS 699 Capstone/Data/imf_gdp_pc_target_values.csv',index=False)
# file_model_df.to_csv('imf_gdp_pc_target_values.csv',index=False)

print(file_model_df)

                        1980      1981      1982      1983      1984  \
Country Name                                                           
Afghanistan            0.000     0.000     0.000     0.000     0.000   
Albania              728.359   817.734   824.542   815.529   788.332   
Algeria             2268.607  2305.505  2254.328  2316.679  2432.717   
Andorra                0.000     0.000     0.000     0.000     0.000   
Angola               745.119   679.020   661.520   671.804   693.779   
...                      ...       ...       ...       ...       ...   
Vietnam              652.593   318.521   414.347   612.565  1044.409   
West Bank and Gaza     0.000     0.000     0.000     0.000     0.000   
Yemen                  0.000     0.000     0.000     0.000     0.000   
Zambia               718.795   718.743   672.013   562.441   448.532   
Zimbabwe               0.000     0.000     0.000     0.000     0.000   

                        1985      1986      1987      1988     

Processing Files in Folders for endogenous variables:

In [43]:
# course_file_path = '/Users/stephenontko/Documents/University of Michigan/UMSI/UMSI MADS/UMSI MADS 2023/SIADS 699 Capstone/Data/IMF/'

#files in folders:
files = [
         'ED_02-07-2023 04-41-04-68_timeSeries/ED_02-07-2023 04-41-04-68_timeSeries.csv'
         ,'FDI_07-14-2022 10-45-25-27_timeSeries/FDI_07-14-2022 10-45-25-27_timeSeries.csv'
         ,'FM_02-07-2023 04-43-40-20_timeSeries/FM_02-07-2023 04-43-40-20_timeSeries.csv'
         ,'FSIRE_02-04-2023 08-23-40-70_timeSeries/FSIRE_02-04-2023 08-23-40-70_timeSeries.csv'
         ,'HPDD_04-11-2020 23-37-06-60_timeSeries/HPDD_04-11-2020 23-37-06-60_timeSeries.csv'
         ] 

#countries or non-countries not in scope
country_exclusion_list = [
'Advanced Economies',
'Advanced Markets',
'Advanced G20',
'Africa',
'Africa (Region)',
'All Countries',
'American Samoa',
'Anguilla',
'Asia and Pacific',
'Australia and New Zealand',
'Belgium-Luxembourg',
'Bermuda',
'Bonaire, St. Eustatius and Saba',
'Bouvet Island',
'British Indian Ocean Territory',
'British Virgin Islands',
'Caribbean',
'CEMAC',
'Central America',
'Central Asia and the Caucasus',
'Central African Economic and Monetary Community',
'CFA franc zone',
'Common Market of Eastern and Southern Africa (COMESA)',
'Cayman Islands',
'Central and South Asia',
'Christmas Island',
'Cocos (Keeling) Islands',
'Cook Islands',
'Curaçao, Kingdom of the Netherlands',
'Curaçao and Sint Maarten',
'Cuba',
'Czechoslovakia',
'East African Community (EAC -5)',
'Economic Community of West African States (ECOWAS)',
'Economies of Persian Gulf',
'East Asia',
'Eastern Caribbean Currency Union',
'Eastern Caribbean Currency Union (ECCU)'
'Eastern Europe ', 
'Emerging G20',
'Emerging Market and Middle-Income Asia ',
'Emerging Market and Middle-Income Economies ',
'Emerging Market and Middle-Income Europe ',
'Emerging Market and Middle-Income Latin America ',
'Emerging Market and Middle-Income Middle East and North Africa and Pakistan',
'Emerging and Developing Countries',
'Emerging and Developing Economies',
'Emerging Markets',
'Europe',
'European Central Bank',
'Faroe Islands',
'French Polynesia',
'French Territories: French Polynesia',
'French Territories: New Caledonia',
'Falkland Islands (Malvinas)',
'French Southern Territories',
'G20',
'G7',
'Gibraltar',
'Greenland',
'Guadeloupe',
'Guernsey',
'Guam',
'Guiana, French',
'Heard Island and McDonald Islands',
'Holy See',
'International Organizations',
'International Organizations and SEFER',
'Isle of Man',
'Jersey',
"Korea, Dem. People's Rep. of",
"Korea, Democratic People's Rep. of",
'Low-Income and Developing Countries',
'Low-Income Developing Asia ',
'Low-Income Developing Latin America',
'Low-Income Developing Others ',
'Low-Income Developing Sub-Saharan Africa',
'Low-Income Oil Producers',
'Liechtenstein',
'Mayotte',
'Monaco',
'Martinique',
'Middle East (Region)',
'Montserrat',
'Netherlands Antilles',
'New Caledonia',
'Niue',
'Norfolk Island',
'North Atlantic and Caribbean',
'North and Central America',
'Northern Mariana Islands',
'Not Specified (including Confidential)',
'North Africa',
'North America',
'Oceania and Polar Regions',
'Other Near and Middle East Economies',
'Pitcairn Islands',
'Pacific Islands ', 
'Reunion',
'Saint Helena',
'Saint Pierre and Miquelon',
'South Georgia and Sandwich Islands',
'Serbia and Montenegro',
'Sint Maarten, Kingdom of the Netherlands',
'South America',
'South Asia',
'Southeast Asia',
'Sub-Saharan Africa (Region) ',
'Tokelau',
'US Pacific Islands',
'United States Virgin Islands',
'Wallis and Futuna Islands',
'Western Sahara',
'Western Europe',
'Western Hemisphere (Region)',
'ASEAN-5',
'Advanced economies',
'Emerging and Developing Asia',
'Emerging and Developing Europe',
'Emerging market and developing economies',
'Emerging and Developing Economies',
'Euro area',
'Euro Area',
'European Union',
'Latin America and the Caribbean',
'Major advanced economies (G7)',
'Middle East and Central Asia',
'Other advanced economies',
'SSA countries in fragile and conflict-affected situations',
'SSA low-income countries',
'SSA low-income countries excluding countries in fragile situations',
'SSA middle-income countries',
'SSA middle-income countries excluding South Africa and Nigeria',
'SSA oil-exporting countries',
'SSA oil-exporting countries excluding Nigeria',
'SSA oil-importing countries',
'SSA oil-importing countries excluding South Africa',
'Southern African Customs Union (SACU)',
'Southern African Development Community (SADC)',
'Sub-Saharan Africa excluding Nigeria and South Africa',
'Sub-saharan Africa',
'Sub-Saharan Africa',
'Swaziland',
'Turks and Caicos Islands',
'Virgin Islands, British',
'West African Economic and Monetary Union (WAEMU)',
'Western Hemisphere',
'World',
'World Minus 25 Significant Financial Centers',
'Yugoslavia',
'nan',
'NaN',
'©IMF, 2022'
]

data_df = pd.DataFrame()

In [44]:
for file in files[:]:
    start_time = time.perf_counter()
    print(file)
#     file_df = pd.read_csv(course_file_path+file)#'/Users/stephenontko/Documents/University of Michigan/UMSI/UMSI MADS/UMSI MADS 2023/SIADS 699 Capstone/Data/imf_gdp_pc_target_values.csv')
    file_df = pd.read_csv(file)
    file_df = file_df[file_df['Attribute']=='Value']
    file_df = file_df[~file_df['Country Name'].isin(country_exclusion_list)]
    
    #edit country names for target value country mapping
    
    file_df=file_df.replace('Afghanistan, Islamic Republic of','Afghanistan')
    file_df=file_df.replace('Afghanistan, Islamic Rep. of','Afghanistan')
    file_df=file_df.replace('Afghanistan, I.R. of','Afghanistan')
    file_df=file_df.replace('Andorra, Principality of','Andorra')
    file_df=file_df.replace('Armenia, Rep. of','Armenia')
    file_df=file_df.replace('Armenia, Republic of','Armenia')
    file_df=file_df.replace('Aruba, Kingdom of the Netherlands','Aruba')
    file_df=file_df.replace('Azerbaijan, Rep. of','Azerbaijan')
    file_df=file_df.replace('Azerbaijan, Republic of','Azerbaijan')
    file_df=file_df.replace('The Bahamas','Bahamas, The')
    file_df=file_df.replace('Bahrain, Kingdom of','Bahrain')
    file_df=file_df.replace('Belarus, Rep. of','Belarus')
    file_df=file_df.replace('Bosnia & Herzegovina','Bosnia and Herzegovina')
    file_df=file_df.replace('Burma ','Burma')
    file_df=file_df.replace('Central African Rep.','Central African Republic')
    file_df=file_df.replace('Equatorial Guinea, Rep. of','Equatorial Guinea')
    file_df=file_df.replace('Eritrea, The State of','Eritrea')
    file_df=file_df.replace('Cape Verde','Cabo Verde')
    file_df=file_df.replace('Comoros, Union of the','Comoros')
    file_df=file_df.replace('Congo, Rep. of','Congo, Republic of ')
    file_df=file_df.replace("Cote d'Ivoire","Côte d'Ivoire")
    file_df=file_df.replace("Democratic Republic of the Congo","Congo, Dem. Rep. of the")
    file_df=file_df.replace("Congo, Democratic Republic of","Congo, Dem. Rep. of the")
    file_df=file_df.replace("Congo, Dem. Rep. of","Congo, Dem. Rep. of the")
    file_df=file_df.replace("Congo, Republic of","Congo, Republic of ")
    file_df=file_df.replace("China","China, People's Republic of")
    file_df=file_df.replace("China,P.R.: Mainland","China, People's Republic of")
    file_df=file_df.replace("China, P.R.: Hong Kong","Hong Kong SAR")
    file_df=file_df.replace("China,P.R.:Hong Kong","Hong Kong SAR")
    file_df=file_df.replace("China, P.R.: Macao","Macao SAR")
    file_df=file_df.replace("China, P.R.: Mainland","China, People's Republic of")
    file_df=file_df.replace('Croatia, Rep. of','Croatia')
    file_df=file_df.replace('Czech Rep.','Czech Republic')
    file_df=file_df.replace('Dominican Rep.','Dominican Republic')
    file_df=file_df.replace('Egypt, Arab Rep. of','Egypt')
    file_df=file_df.replace('Estonia, Rep. of','Estonia')
    file_df=file_df.replace('Ethiopia, The Federal Dem. Rep. of','Ethiopia')
    file_df=file_df.replace('Fiji, Rep. of','Fiji')
    file_df=file_df.replace("Eswatini, Kingdom of","Eswatini")
    file_df=file_df.replace('Iran, Islamic Rep. of','Iran')
    file_df=file_df.replace('Iran, Islamic Republic of','Iran')
    file_df=file_df.replace('Iran, I.R. of','Iran')
    file_df=file_df.replace('Kazakhstan, Rep. of','Kazakhstan')
    file_df=file_df.replace('Korea, Rep. of','Korea, Republic of')
    file_df=file_df.replace('Korea','Korea, Republic of')
    file_df=file_df.replace('Kosovo, Rep. of','Kosovo')
    file_df=file_df.replace('Kosovo, Republic of','Kosovo')
    file_df=file_df.replace('Kyrgyz Rep.','Kyrgyz Republic')
    file_df=file_df.replace("Lao People's Dem. Rep.",'Lao P.D.R.')
    file_df=file_df.replace("Lao People's Dem.Rep",'Lao P.D.R.')
    file_df=file_df.replace('Lesotho, Kingdom of','Lesotho')
    file_df=file_df.replace("Lao People's Democratic Republic","Lao P.D.R.")
    file_df=file_df.replace("Marshall Islands, Republic of","Marshall Islands")
    file_df=file_df.replace("Micronesia, Federated States of","Micronesia, Fed. States of")
    file_df=file_df.replace("Micronesia","Micronesia, Fed. States of")
    file_df=file_df.replace('Madagascar, Rep. of','Madagascar')
    file_df=file_df.replace('Marshall Islands, Rep. of the','Marshall Islands')
    file_df=file_df.replace('Mauritania, Islamic Rep. of','Mauritania')
    file_df=file_df.replace('Moldova, Rep. of','Moldova')
    file_df=file_df.replace('Montenegro, Rep. of','Montenegro')
    file_df=file_df.replace('Mozambique, Rep. of','Mozambique')
    file_df=file_df.replace('Myanmar','Burma')
    file_df=file_df.replace('Nauru, Rep. of','Nauru')
    file_df=file_df.replace('Netherlands, The','Netherlands')
    file_df=file_df.replace('North Macedonia, Republic of','North Macedonia ')
    file_df=file_df.replace('North Macedonia, Republic of ','North Macedonia ')
    file_df=file_df.replace('North Macedonia','North Macedonia ')
    file_df=file_df.replace('Palau, Rep. of','Palau')
    file_df=file_df.replace('Poland, Rep. of','Poland')
    file_df=file_df.replace('Russia','Russian Federation')
    file_df=file_df.replace('San Marino, Rep. of','San Marino')
    file_df=file_df.replace('St. Kitts and Nevis ','Saint Kitts and Nevis')
    file_df=file_df.replace('St. Kitts and Nevis','Saint Kitts and Nevis')
    file_df=file_df.replace('St. Lucia','Saint Lucia')
    file_df=file_df.replace('St. Vincent and the Grenadines','Saint Vincent and the Grenadines')
    file_df=file_df.replace('St. Vincent & Grens.','Saint Vincent and the Grenadines')
    file_df=file_df.replace('St. Lucia','Saint Lucia')
    file_df=file_df.replace("Sao Tome and Principe","São Tomé and Príncipe")
    file_df=file_df.replace("Sao Tome & Principe","São Tomé and Príncipe")
    file_df=file_df.replace("South Sudan","South Sudan, Republic of")
    file_df=file_df.replace('Serbia, Rep. of','Serbia')
    file_df=file_df.replace('Serbia, Republic of','Serbia')
    file_df=file_df.replace('Slovak Rep.','Slovak Republic')
    file_df=file_df.replace('Slovenia, Rep. of','Slovenia')
    file_df=file_df.replace('South Sudan, Rep. of','South Sudan, Republic of')
    file_df=file_df.replace('Syrian Arab Rep.','Syria')
    file_df=file_df.replace('Syrian Arab Republic','Syria')
    file_df=file_df.replace('São Tomé and Príncipe, Dem. Rep. of','São Tomé and Príncipe')
    file_df=file_df.replace("Timor-Leste, Dem. Rep. of","Timor-Leste")
    file_df=file_df.replace('Tajikistan, Rep. of','Tajikistan')
    file_df=file_df.replace('Taiwan Province of China','Taiwan')
    file_df=file_df.replace('Taiwan Prov.of China','Taiwan')
    file_df=file_df.replace('Tanzania, United Rep. of','Tanzania')
    file_df=file_df.replace('Türkiye, Rep of','Türkiye, Republic of')
    file_df=file_df.replace('Turkey','Türkiye, Republic of')
    file_df=file_df.replace('Uzbekistan, Rep. of','Uzbekistan')
    file_df=file_df.replace('Venezuela, Rep. Bolivariana de','Venezuela')
    file_df=file_df.replace('Venezuela, Republica Bolivariana de','Venezuela')
    file_df=file_df.replace('Venezuela, Rep. Bol.','Venezuela')
    file_df=file_df.replace('Yemen Arab Rep.','Yemen')
    file_df=file_df.replace("Yemen, People's Dem. Rep. of",'Yemen')
    file_df=file_df.replace('Yemen, Rep. of','Yemen')
    file_df=file_df.replace('Yemen, Republic of','Yemen')
    
    print(file_df)
    print(file_df.columns)
    indicator_codes = file_df['Indicator Code'].unique()
    print(indicator_codes)
    # print(file_df[file_df['Indicator Code']==indicator_codes[0]])
    # print(file_df[file_df['Indicator Code']==indicator_codes[1]])
    # print(file_df[file_df['Indicator Code']==indicator_codes[2]])
    folder_df = pd.DataFrame(columns=['Country Name','Country Code','Indicator Name'
                                      ,'Indicator Code','Year','Feature Value'])
    for ind_code in indicator_codes[:]:
        print(ind_code)
        var_df = file_df[file_df['Indicator Code']==ind_code]
        print(var_df.columns)    
        years = []
        for col in var_df.columns:
            try:
                if type(int(col)) is int:
                # print(col)
                    years.append(col)
            except:
                continue
        print(years)
        if file == 'FSIRE_02-04-2023 08-23-40-70_timeSeries/FSIRE_02-04-2023 08-23-40-70_timeSeries.csv':
            var_df = var_df[['Country Name','Country Code','Indicator Name','Indicator Code','Financial Institution Code']+years]
            var_df['Indicator Code'] = var_df['Indicator Code']+'_'+var_df['Financial Institution Code']
            var_df = var_df[['Country Name','Country Code','Indicator Name','Indicator Code']+years]
            var_df=var_df.set_index(['Country Name', 'Country Code','Indicator Name','Indicator Code']).stack().reset_index()
            var_df.rename(columns={var_df.columns[-2]:'Year',var_df.columns[-1]:'Feature Value'},inplace=True)
            var_df['Feature Value'] = var_df['Feature Value'].astype(float)
            var_df=var_df.groupby(['Country Name','Country Code','Indicator Name','Indicator Code','Year'],as_index = False).agg({'Feature Value': np.mean})#.reset_index()#['Feature Value'].sum().reset_index()
            print(var_df)
        else:
            var_df = var_df[['Country Name','Country Code','Indicator Name','Indicator Code']+years]
            var_df=var_df.set_index(['Country Name', 'Country Code','Indicator Name','Indicator Code']).stack().reset_index()
            var_df.rename(columns={var_df.columns[-2]:'Year',var_df.columns[-1]:'Feature Value'},inplace=True)
        print(var_df)
        print(var_df.columns)
        # dt_col_results = dt_col_results.append(iter_df)
        folder_df = folder_df.append(var_df)
        
#     folder_df.to_csv(course_file_path+file.split('/')[0]+'/'+file.split('_')[0]+'_df.csv',index=True)
#     folder_df.to_csv(file.split('/')[0]+'/'+file.split('_')[0]+'_df.csv',index=True)
    print(var_df)
    end_time = time.perf_counter()
    print('Process time: ',f"{end_time-start_time:0.4f} seconds {(end_time-start_time)/60:0.4f} minutes.")

ED_02-07-2023 04-41-04-68_timeSeries/ED_02-07-2023 04-41-04-68_timeSeries.csv
    Country Name  Country Code                Indicator Name Indicator Code  \
0          Aruba           314              Extensive Margin  between_theil   
1          Aruba           314              Intensive Margin   within_theil   
2          Aruba           314  Export Diversification Index    total_theil   
3    Afghanistan           512              Extensive Margin  between_theil   
4    Afghanistan           512              Intensive Margin   within_theil   
..           ...           ...                           ...            ...   
592    Guatemala           258  Export Diversification Index    total_theil   
593     Paraguay           288  Export Diversification Index    total_theil   
597        Qatar           453              Extensive Margin  between_theil   
598        Qatar           453              Intensive Margin   within_theil   
599        Qatar           453  Export Diversificatio

/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folde

FD_FD_IX
Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       'Attribute', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', 'Unnamed: 46'],
      dtype='object')
['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
     Country Name Country Code               Indicator Name Indicator Code  \
0     Saint Lucia          362  Financial Development Index       FD_FD_IX   
1     Saint Lucia

/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folde

     Country Name Country Code          Indicator Name  Indicator Code  Year  \
0      Azerbaijan          912  Expenditure (% of GDP)  G_X_G01_GDP_PT  1995   
1      Azerbaijan          912  Expenditure (% of GDP)  G_X_G01_GDP_PT  1996   
2      Azerbaijan          912  Expenditure (% of GDP)  G_X_G01_GDP_PT  1997   
3      Azerbaijan          912  Expenditure (% of GDP)  G_X_G01_GDP_PT  1998   
4      Azerbaijan          912  Expenditure (% of GDP)  G_X_G01_GDP_PT  1999   
...           ...          ...                     ...             ...   ...   
6309      Somalia          726  Expenditure (% of GDP)  G_X_G01_GDP_PT  2023   
6310      Somalia          726  Expenditure (% of GDP)  G_X_G01_GDP_PT  2024   
6311      Somalia          726  Expenditure (% of GDP)  G_X_G01_GDP_PT  2025   
6312      Somalia          726  Expenditure (% of GDP)  G_X_G01_GDP_PT  2026   
6313      Somalia          726  Expenditure (% of GDP)  G_X_G01_GDP_PT  2027   

      Feature Value  
0         18.6651

...

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       'Financial Institution Name', 'Financial Institution Code', 'Attribute',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022', 'Unnamed: 30'],
      dtype='object')
['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
            Country Name  Country Code  \
0                Albania           914   
1                Albania           914   
2                Albania           914   
3                Albania           914   
4                Albania           914   
...                  ...           ...   
4440             Vietnam           582   
4441             Vietnam           582   
4442             Vietnam           582 

/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folde

            Country Name  Country Code  \
0                Algeria           612   
1                Algeria           612   
2                Algeria           612   
3                Algeria           612   
4                Algeria           612   
...                  ...           ...   
2246  West Bank and Gaza           487   
2247  West Bank and Gaza           487   
2248  West Bank and Gaza           487   
2249  West Bank and Gaza           487   
2250  West Bank and Gaza           487   

                                         Indicator Name     Indicator Code  \
0     Total assets foreign bank branches, Domestic C...    FA_XDC_FS_ODX_D   
1     Total assets foreign bank branches, Domestic C...    FA_XDC_FS_ODX_D   
2     Total assets foreign bank branches, Domestic C...    FA_XDC_FS_ODX_D   
3     Total assets foreign bank branches, Domestic C...    FA_XDC_FS_ODX_D   
4     Total assets foreign bank branches, Domestic C...    FA_XDC_FS_ODX_I   
...                        

                   Country Name  Country Code     Indicator Name  \
0                      Portugal           182  Debt to GDP Ratio   
1                         Qatar           453  Debt to GDP Ratio   
2                       Romania           968  Debt to GDP Ratio   
3            Russian Federation           922  Debt to GDP Ratio   
4                        Rwanda           714  Debt to GDP Ratio   
..                          ...           ...                ...   
186                       Tonga           866  Debt to GDP Ratio   
187  Micronesia, Fed. States of           868  Debt to GDP Ratio   
188                    Mongolia           948  Debt to GDP Ratio   
189                 Afghanistan           512  Debt to GDP Ratio   
190                 Puerto Rico           359  Debt to GDP Ratio   

    Indicator Code Attribute  1800  1801  1802  1803  1804  ...       2007  \
0       GGXWDG_GDP     Value   NaN   NaN   NaN   NaN   NaN  ...  68.439083   
1       GGXWDG_GDP     Valu

/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append(var_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/818379670.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folde

Combine All Features From All Files Into Single Dataframe/File

In [45]:

feature_df = pd.DataFrame(columns=['Country Name','Country Code','Indicator Name'
                                      ,'Indicator Code','Year','Feature Value','filename'])
for file_name in files:
    start_time = time.perf_counter()
    print(file_name)
#     print(course_file_path+file_name.split('/')[0]+'/'+file_name.split('_')[0])
#     df_file = pd.read_csv(course_file_path+file_name.split('/')[0]+'/'+file_name.split('_')[0]+'_df.csv')
    df_file = pd.read_csv(file_name.split('/')[0]+'/'+file_name.split('_')[0]+'_df.csv')
    print(df_file)
    print(df_file.columns)
#     df_file = df_file[['Country Name','Country Code','Indicator Name','Indicator Code','Year','Feature Value','filename']]
    df_file = df_file[['Country Name','Country Code','Indicator Name','Indicator Code','Year','Feature Value']]
    print(df_file)
    print(df_file.columns)
    feature_df = feature_df.append(df_file)
    # print(course_file_path+df_file.split('/')[0]+'/metadata_'+df_file.split('/')[1])
    # meta_of_df = pd.read_csv(course_file_path+df_file.split('/')[0]+'/metadata_'+df_file.split('/')[1])
    # meta_of_df = meta_of_df[meta_of_df['Country Name'].isnull() & meta_of_df['Metadata Attribute'].isin(['Dataset','Definition'])].iloc[:,7:8].head()
    # print(meta_of_df)

    # folder_df.to_csv(course_file_path+file.split('/')[0]+'/'+file.split('_')[0]+'_df.csv',index=True)
    folder_df.to_csv(file.split('/')[0]+'/'+file.split('_')[0]+'_df.csv',index=True)


    end_time = time.perf_counter()
    print('Process time: ',f"{end_time-start_time:0.4f} seconds {(end_time-start_time)/60:0.4f} minutes.")

# feature_df.to_csv('feature_yearly_data_'+str(date.today())+'.csv',index=True)
# feature_df.to_csv('feature_yearly_data_2023-04-01.csv',index=True)


print(feature_df)

ED_02-07-2023 04-41-04-68_timeSeries/ED_02-07-2023 04-41-04-68_timeSeries.csv
       Unnamed: 0 Country Name  Country Code                Indicator Name  \
0               0        Aruba           314              Extensive Margin   
1               1        Aruba           314              Extensive Margin   
2               2        Aruba           314              Extensive Margin   
3               3        Aruba           314              Extensive Margin   
4               4        Aruba           314              Extensive Margin   
...           ...          ...           ...                           ...   
29251        9747        Qatar           453  Export Diversification Index   
29252        9748        Qatar           453  Export Diversification Index   
29253        9749        Qatar           453  Export Diversification Index   
29254        9750        Qatar           453  Export Diversification Index   
29255        9751        Qatar           453  Export Diversifica

/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/2877803553.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_df = feature_df.append(df_file)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/2877803553.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_df = feature_df.append(df_file)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/2877803553.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_df = feature_df.append(df_file)


Process time:  0.0755 seconds 0.0013 minutes.
FSIRE_02-04-2023 08-23-40-70_timeSeries/FSIRE_02-04-2023 08-23-40-70_timeSeries.csv
       Unnamed: 0   Country Name  Country Code  \
0               0        Albania           914   
1               1        Albania           914   
2               2        Albania           914   
3               3        Algeria           612   
4               4        Algeria           612   
...           ...            ...           ...   
76779           0  United States           111   
76780           1  United States           111   
76781           2  United States           111   
76782           3  United States           111   
76783           4  United States           111   

                                          Indicator Name       Indicator Code  \
0      Number of domestic subsidiaries of domesticall...     DSD_NUM_FS_OFX_I   
1      Number of domestic subsidiaries of domesticall...   DSD_NUM_FS_OFX_IFA   
2      Number of domestic 

/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/2877803553.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_df = feature_df.append(df_file)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/2877803553.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_df = feature_df.append(df_file)


In [46]:

def process_data_for_model(train_year, inputdate):
    print('training year: ',train_year, ' date: ',inputdate)
#     data_df = pd.read_csv(course_file_path+'feature_yearly_data_'+str(inputdate)+'.csv')
    data_df = pd.read_csv('feature_yearly_data_'+str(inputdate)+'.csv')
    data_df=data_df[['Country Name','Indicator Code','Year','Feature Value']]
    train_data_df = data_df[data_df['Year'] <= train_year]
    train_data_df = train_data_df[['Country Name','Indicator Code','Feature Value']]
    train_data_df = pd.pivot_table(train_data_df, values = 'Feature Value'
                                   ,index = ['Country Name']
                                   ,columns = ['Indicator Code']
                                   ,aggfunc=np.mean).fillna(0).astype(float)
    print(train_data_df)
    print(train_data_df.columns)

#     train_data_df.to_csv(course_file_path+'train_feature_data_'+str(train_year)+'_'+str(inputdate)+'.csv',index=True)
#     train_data_df.to_csv('train_feature_data_'+str(train_year)+'_'+str(inputdate)+'.csv',index=True)

    
    test_data_df = data_df[data_df['Year'] <= train_year+1]
    test_data_df = test_data_df[['Country Name','Indicator Code','Feature Value']]
    test_data_df = test_data_df[['Country Name','Indicator Code','Feature Value']]
    test_data_df = pd.pivot_table(test_data_df, values = 'Feature Value'
                                   ,index = ['Country Name']
                                   ,columns = ['Indicator Code']
                                   ,aggfunc=np.mean).fillna(0).astype(float)
#     test_data_df.to_csv(course_file_path+'test_feature_data_'+str(train_year+1)+'_'+str(date.today())+'.csv',index=True)
#     test_data_df.to_csv('test_feature_data_'+str(train_year+1)+'_'+str(inputdate)+'.csv',index=True)
    
train_year = 2020
process_data_for_model(train_year, '2023-04-01')

training year:  2020  date:  2023-04-01
Indicator Code      DA_EUR_FS_ODX_D  DA_EUR_FS_ODX_I  DA_EUR_FS_ODX_ID  \
Country Name                                                             
Afghanistan                     0.0              0.0               0.0   
Albania                         0.0              0.0               0.0   
Algeria                         0.0              0.0               0.0   
Andorra                         0.0              0.0               0.0   
Angola                          0.0              0.0               0.0   
...                             ...              ...               ...   
Vietnam                         0.0              0.0               0.0   
West Bank and Gaza              0.0              0.0               0.0   
Yemen                           0.0              0.0               0.0   
Zambia                          0.0              0.0               0.0   
Zimbabwe                        0.0              0.0               0.0  

In [47]:
#prepare training data

train_model_df = file_model_df.copy()
test_model_df = file_model_df.copy()

def process_data_for_vif(train_year, inputdate):
    print('training year: ',train_year, ' date: ',inputdate)
    
    exclude_column_list = ['Xgdppc','PPPGDP']
    
    exclude_list = ['European Central Bank']

    train_y=train_model_df[[train_year]].replace('no data',0).fillna(0)
    test_y=test_model_df[[train_year+1]].replace('no data',0).fillna(0)

#     train_X = pd.read_csv(course_file_path+'train_feature_data_'+str(train_year)+'_'+str(inputdate)+'.csv')
    train_X = pd.read_csv('train_feature_data_'+str(train_year)+'_'+str(inputdate)+'.csv')
    train_X.index = train_X['Country Name']
    train_X = train_X.drop(columns =['Country Name'])
    train_X = train_X[~train_X.index.isin(exclude_list)]
    print(train_X)
    print(train_X.columns)
#     test_X = pd.read_csv(course_file_path+'test_feature_data_'+str(train_year+1)+'_'+str(inputdate)+'.csv')
    test_X = pd.read_csv('test_feature_data_'+str(train_year+1)+'_'+str(inputdate)+'.csv')
    test_X.index = test_X['Country Name']
    test_X = test_X.drop(columns =['Country Name'])
    test_X = test_X[~test_X.index.isin(exclude_list)]
    print(test_X)
    print(test_X.columns)
    code_names = train_X.columns
    
    return train_X, code_names#, train_y, test_X, test_y

train_year = 2020
filedate = '2023-04-01'

# train_X, train_y, test_X, test_y, code_names, filename =  process_data_for_model(train_year, filedate)
train_X, code_names =  process_data_for_vif(train_year, filedate)

training year:  2020  date:  2023-04-01
                    DA_EUR_FS_ODX_D  DA_EUR_FS_ODX_I  DA_EUR_FS_ODX_ID  \
Country Name                                                             
Afghanistan                     0.0              0.0               0.0   
Albania                         0.0              0.0               0.0   
Algeria                         0.0              0.0               0.0   
Andorra                         0.0              0.0               0.0   
Angola                          0.0              0.0               0.0   
...                             ...              ...               ...   
Vietnam                         0.0              0.0               0.0   
West Bank and Gaza              0.0              0.0               0.0   
Yemen                           0.0              0.0               0.0   
Zambia                          0.0              0.0               0.0   
Zimbabwe                        0.0              0.0               0.0  

Measuring Multi-Collinearity for Endogenous Variables

In [48]:
def calc_variance_inflation_factor(train_X):
    train_X = train_X.dropna()
    train_X = add_constant(train_X)
    train_X_scaled = StandardScaler().fit(train_X).transform(train_X)
    train_X_scaled = add_constant(train_X_scaled)
    # train_X_scaled = MinMaxScaler().fit(train_X).transform(train_X)
    vif = pd.DataFrame()
    vif['features'] = train_X.columns
    vif['feature_sums'] = [x for x in train_X.sum(axis=0)]
    vif = vif[vif['feature_sums']!=0]
    train_X = train_X[vif['features']]
    vif['VIF'] = [variance_inflation_factor(train_X, i) for i in range(train_X.shape[1])]
    return(vif)

vif_df = calc_variance_inflation_factor(train_X)
vif_df = vif_df.sort_values(by=['VIF'], ascending=False).round(4)

print(vif_df)

# vif_df.to_csv(data_model_file_path+'vif_train_model_df_'+str(train_year)+'_'+str(filedate)+'.csv', index=True)
# vif_df.to_csv('vif_train_model_df_'+str(train_year)+'_'+str(filedate)+'.csv', index=True)



/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


               features  feature_sums     VIF
376    FA_XDC_FS_ODX_XI  4.637529e+12     inf
502   IA_EUR_FS_OFX_IIC  9.321684e+09     inf
518   IA_USD_FS_OFX_IIC  9.193971e+10     inf
515    IA_USD_FS_ODX_XO  1.928200e+10     inf
514  IA_USD_FS_ODX_XMFI  7.869977e+09     inf
..                  ...           ...     ...
424           FD_FMA_IX  2.997170e+01  1.9839
833        within_theil  4.774708e+02  1.7598
422           FD_FIE_IX  9.277000e+01  1.3456
832         total_theil  5.786679e+02  1.1058
0                 const  1.960000e+02  0.0000

[536 rows x 3 columns]


In [49]:

iso_country_codes = {
    'Afghanistan':4,
'Albania':8,
'Algeria':12,
'American Samoa':16,
'Andorra':20,
'Angola':24,
'Antigua and Barbuda':28,
'Argentina':32,
    'Armenia':51,
    'Aruba':533,
'Australia':36,
'Austria':40,
    'Azerbaijan':31,
    'Bahamas, The':44,
    'Bahrain':48,
'Bangladesh':50,
    'Barbados':52,
    'Belarus':112,
'Belgium':56,
    'Belize':84,
    'Benin':204,
'Bermuda':60,
'Bhutan':64,
'Bolivia':68,
'Bosnia and Herzegovina':70,
'Botswana':72,
    'Brazil':76,
    'Brunei Darussalam':96,
'Bulgaria':100,
    'Burkina Faso':854,
    'Burma':104,
    'Burundi':108,
    'Cabo Verde':132,
'Cambodia':116,
'Cameroon':120,
    'Canada':124,
    'Central African Republic':140,
    'Chad':148,
    'Chile':152,
    "China, People's Republic of":156,
'Colombia':170,
'Comoros':174,
'Congo, Republic of ':178,
'Congo, Dem. Rep. of the':180,
    'Costa Rica':188,
    "Côte d'Ivoire":384,
'Croatia':191,
'Cuba':192,
'Cyprus':196,
'Czech Republic':203,
    'Denmark':208,
    'Djibouti':262,
'Dominica':212,
'Dominican Republic':214,
'Ecuador':218,
    'Egypt':818,
'El Salvador':222,
'Equatorial Guinea':226,
'Eritrea':232,
'Estonia':233,
    'Eswatini':748,
    'Ethiopia':231,
'Falkland Islands':238,
'Faroe Islands':234,
'Fiji':242,
    'Finland':246,
    'France':250,
'Gabon':266,
'Gambia, The':270,
'Georgia':268,
    'Germany':276,
    'Ghana':288,
    'Greece':300,
'Grenada':308,
'Guadeloupe':312,
'Guam':316,
'Guatemala':320,
'Guernsey':831,
'Guinea':324,
'Guinea-Bissau':624,
'Guyana':328,
    'Haiti':332,
    'Honduras':340,
    'Hong Kong SAR':344,
    'Hungary':348,
    'Iceland':352,
    'India':356,
'Indonesia':360,
'Iran':364,
'Iraq':368,
'Ireland':372,
'Israel':376,
'Italy':380,
'Jamaica':388,
'Japan':392,
'Jordan':400,
'Kazakhstan':398,
'Kenya':404,
'Kiribati':296,
"Korea (Democratic People's Republic of)":408,
"Korea, Republic of":410,
    'Kosovo':None,
'Kuwait':414,
'Kyrgyz Republic':417,
"Lao P.D.R.":418,
'Latvia':428,
'Lebanon':422,
'Lesotho':426,
'Liberia':430,
'Libya':434,
'Liechtenstein':438,
'Lithuania':440,
'Luxembourg':442,
'Macao SAR':446,
'Madagascar':450,
'Malawi':454,
'Malaysia':458,
'Maldives':462,
'Mali':466,
'Malta':470,
'Marshall Islands':584,
'Mauritania':478,
'Mauritius':480,
'Mexico':484,
'Micronesia, Fed. States of':583,
'Moldova':498,
'Monaco':492,
'Mongolia':496,
'Montenegro':499,
'Montserrat':500,
'Morocco':504,
'Mozambique':508,
'Namibia':516,
'Nauru':520,
'Nepal':524,
'Netherlands':528,
'New Zealand':554,
'Nicaragua':558,
'Niger':562,
'Nigeria':566,
'Niue':570,
'Norfolk Island':574,
'North Macedonia ':807,
'Northern Mariana Islands':580,
'Norway':578,
    'Oman':512,
'Pakistan':586,
'Palau':585,
'Panama':591,
'Papua New Guinea':598,
'Paraguay':600,
'Peru':604,
'Philippines':608,
'Pitcairn':612,
'Poland':616,
'Portugal':620,
'Puerto Rico':630,
'Qatar':634,
'Romania':642,
'Russian Federation':643,
    'Rwanda':646,
    'Saint Kitts and Nevis':659,
'Saint Lucia':662,
'Saint Martin':663,
'Saint Pierre and Miquelon':666,
'Saint Vincent and the Grenadines':670,
'Samoa':882,
'San Marino':674,
'São Tomé and Príncipe':678,
'Saudi Arabia':682,
'Senegal':686,
'Serbia':688,
'Seychelles':690,
'Sierra Leone':694,
'Singapore':702,
'Slovak Republic':703,
'Slovenia':705,
'Solomon Islands':90,
'Somalia':706,
'South Africa':710,
'South Georgia':239,
    'South Sudan, Republic of':728,
    'Spain':724,
'Sri Lanka':144,
'Sudan':729,
'Suriname':740,
'Sweden':752,
'Switzerland':756,
'Syria':760,
'Taiwan':158,
'Tajikistan':762,
'Tanzania':834,
'Thailand':764,
'Timor-Leste':626,
'Togo':768,
'Tonga':776,
'Trinidad and Tobago':780,
'Tunisia':788,
'Türkiye, Republic of':792,
'Turkmenistan':795,
'Tuvalu':798,
'Uganda':800,
'Ukraine':804,
'United Arab Emirates':784,
'United Kingdom':826,
'United States':840,
'Uruguay':858,
'Uzbekistan':860,
'Vanuatu':548,
'Venezuela':862,
'Vietnam':704,
    'West Bank and Gaza':275,
'Yemen':887,
'Zambia':894,
'Zimbabwe':716
    # ,'nan':None
}


In [50]:
def process_data_for_model(train_year, inputdate):
    print('training year: ',train_year, ' date: ',inputdate)
    train_y=train_model_df[[train_year]].replace('no data',0).fillna(0)
    test_y=test_model_df[[train_year+1]].replace('no data',0).fillna(0)

    exclude_list = [ 
        'Eastern Caribbean Currency Union (ECCU)',
        'European Central Bank'
    ]
#     train_X = pd.read_csv(course_file_path+'train_feature_data_'+str(train_year)+'_'+str(inputdate)+'.csv')
    train_X = pd.read_csv('train_feature_data_'+str(train_year)+'_'+str(inputdate)+'.csv')
    train_X.index = train_X['Country Name']
    train_X = train_X.drop(columns =['Country Name'])
    train_X = train_X[~train_X.index.isin(exclude_list)]
    print(train_X)
    print(train_X.columns)
#     test_X = pd.read_csv(course_file_path+'test_feature_data_'+str(train_year+1)+'_'+str(inputdate)+'.csv')
    test_X = pd.read_csv('test_feature_data_'+str(train_year+1)+'_'+str(inputdate)+'.csv')
    test_X.index = test_X['Country Name']
    test_X = test_X.drop(columns =['Country Name'])
    test_X = test_X[~test_X.index.isin(exclude_list)]
    print(test_X)
    print(test_X.columns)
    code_names = train_X.columns
    
    return train_X, train_y, test_X, test_y, code_names

# train_year = 2020
# filedate = '2023-04-01'

# train_X, train_y, test_X, test_y, code_names =  process_data_for_model(train_year, filedate)


In [78]:
# metadata_df = pd.read_csv(course_file_path+'/file_metadata_df.csv')


metadata_df = pd.DataFrame(columns = ['file','file_description','column_code','column_name'])

def file_metadata_processing(metafile,metadata_df):
    print(metafile)
    meta_file_df = pd.read_csv(metafile.split('/')[0]+'/metadata_'+metafile.split('/')[1])
    meta_file_description = meta_file_df[meta_file_df['Country Name'].isnull() & meta_file_df['Metadata Attribute'].isin(['Dataset','Definition'])].iloc[:,7:8].head()
    print(meta_file_description)
#     print(meta_file_df)
#     print(meta_file_df.columns)
    try:
#         print(metafile)
#         print(meta_file_df)
#         print(meta_file_df.columns)
        meta_file_df = meta_file_df[['Indicator Name','Indicator Code','Metadata Attribute','Metadata Value']]
        column_names = meta_file_df['Indicator Name'].unique()
        code_names = meta_file_df['Indicator Code'].unique()
    except:
        print(metafile)
        print(meta_file_df)
        print(meta_file_df.columns)
        if metafile == 'HPDD_04-11-2020 23-37-06-60_timeSeries/HPDD_04-11-2020 23-37-06-60_timeSeries.csv':
            code_names=meta_file_df[meta_file_df['Metadata Attribute']=='Indicator Code']['Metadata Value'].unique()
            column_names=meta_file_df[meta_file_df['Metadata Attribute']=='Indicator Name']['Metadata Value'].unique()
        else:
            meta_file_df = meta_file_df[['Indicator Name','Indicator Source Code','Metadata Attribute','Metadata Value']]
            column_names = meta_file_df['Indicator Name'].unique()
            code_names = meta_file_df['Indicator Source Code'].unique()
#     file_columns[metafile] = [column_names, code_names]
    print(meta_file_df)
    print(meta_file_df.columns)
    print(column_names)
    print(code_names)
    print(len(column_names))
    for colm,codenam in zip(column_names, code_names):
        print(colm,codenam )
        # file_column_names_df.append()
        appendrow = []
        metadata_df = pd.concat([metadata_df,pd.DataFrame([[metafile,meta_file_description,codenam,colm ]]
                                      ,columns = ['file','file_description','column_code','column_name'])])
    return metadata_df

for metafile in files[:]:
    print(metafile)
    metadata_df=file_metadata_processing(metafile,metadata_df)


# print(file_column_names_df)
metadata_df.to_csv('file_metadata_df.csv', index=True)

ED_02-07-2023 04-41-04-68_timeSeries/ED_02-07-2023 04-41-04-68_timeSeries.csv
ED_02-07-2023 04-41-04-68_timeSeries/ED_02-07-2023 04-41-04-68_timeSeries.csv
                                      Metadata Value
1                 Export Diversification and Quality
2  Covers 200 countries with data from 1962 to 20...
        Indicator Name Indicator Code    Metadata Attribute  \
0                  NaN            NaN              Category   
1                  NaN            NaN               Dataset   
2                  NaN            NaN            Definition   
3                  NaN            NaN             Frequency   
4                  NaN            NaN   Geographic Coverage   
...                ...            ...                   ...   
1827  Intensive Margin   within_theil   Indicator Full Name   
1828  Intensive Margin   within_theil        Indicator Name   
1829  Intensive Margin   within_theil   Indicator SDMX Code   
1830  Intensive Margin   within_theil   Indicator SDMX 

Creating Decision Tree Model Function
Model within a model is also being processed - the first decition tree is used to evaluated the significant 
features of the data, which is then fed into the second decision tree for more honed model predition

In [80]:
dt_col_results = pd.DataFrame(columns = ['filename', 'train_year', 'first_model_score', 'second_model_score', 'max_features','max_depth','ccp_alpha','percent_threshold','column','column_significance', 'column_descr'])

def dt_w_sig_cols(train_year, date, max_features, max_depth, ccp_alpha, percent_threshold,iteration):

    train_X, train_y, test_X, test_y, code_names =  process_data_for_model(train_year, filedate)


    feature_list = [x for x in train_X.columns]
    feature_list = [x for x in feature_list if x in test_X.columns]

    train_X = train_X[feature_list].dropna()
    test_X = test_X[feature_list].dropna()

    random_state = 0
    max_features = 'log2'
    max_depth = 100
    
    dt_init_regr = DecisionTreeRegressor(max_features= max_features, max_depth= max_depth, ccp_alpha = ccp_alpha).fit(train_X, train_y)
    dt_regr_predict = dt_init_regr.predict(test_X)
    dt_init_regr_score = dt_init_regr.score(test_X,test_y)
    
    significant_score_indicies =[list(dt_init_regr.feature_importances_).index(x) for x in dt_init_regr.feature_importances_ if x > percent_threshold]
    sig_list = []
    for sig_i, sig_column in enumerate([code_names[x] for x in [list(dt_init_regr.feature_importances_).index(x) for x in dt_init_regr.feature_importances_ if x > percent_threshold]]):
        sig_col_score = round([dt_init_regr.feature_importances_[x] for x in [list(dt_init_regr.feature_importances_).index(x) for x in dt_init_regr.feature_importances_ if x > percent_threshold]][sig_i]*100,6)
        try:
            sig_col_meta_descr = metadata_df[metadata_df['column_code']==sig_column]['column_name'].unique()[0]
        except:
            sig_col_meta_descr = None
        sig_list.append((sig_col_score ,sig_column, sig_col_meta_descr))
    sig_list = sorted(sig_list)
    
    train_X = train_X.iloc[:,significant_score_indicies]
    test_X = test_X.iloc[:,significant_score_indicies]

    print('second data model for significant columns')
    dt_regr2 = DecisionTreeRegressor(max_features= max_features,max_depth= max_depth, ccp_alpha = ccp_alpha).fit(train_X, train_y)
    dt_regr_predict2 = dt_regr2.predict(test_X)    
    dt_regr_score2 = dt_regr2.score(test_X,test_y)
    for sig_i, sig_column in enumerate([code_names[x] for x in [list(dt_init_regr.feature_importances_).index(x) for x in dt_init_regr.feature_importances_ if x > percent_threshold]]):
        sig_col_score = round([dt_init_regr.feature_importances_[x] for x in [list(dt_init_regr.feature_importances_).index(x) for x in dt_init_regr.feature_importances_ if x > percent_threshold]][sig_i]*100,6)
        try:
            sig_col_meta_descr = metadata_df[metadata_df['column_code']==sig_column]['column_name'].unique()[0]
        except:
            sig_col_meta_descr = None
        iter_df = pd.DataFrame([[train_year,dt_regr_score2, dt_init_regr_score, max_features, max_depth, ccp_alpha, percent_threshold, sig_column ,sig_col_score,sig_col_meta_descr, iteration]]
                               ,columns = ['train_year', 'first_model_score', 'second_model_score', 'max_features','max_depth','ccp_alpha','percent_threshold','column','column_significance', 'column_descr','iteration'])
        global dt_col_results
        dt_col_results = dt_col_results.append(iter_df)
        
    return dt_regr2, train_year, test_y, dt_regr_predict2, dt_regr_score2, max_features, max_depth, ccp_alpha, dt_init_regr_score

    

Dataframe Processing to input Data into Choropleth World Map

In [81]:
def choropleth_dfs(test_y, train_year, predictions):
    choropleth_df = test_y.copy().astype(float)
    choropleth_df['Country Name'] = choropleth_df.index
    choropleth_df['GDP Per Capita'] = [x for x in predictions]
    choropleth_df['id'] = [iso_country_codes[x] for x in choropleth_df.index]

    choropleth_actual_df = choropleth_df[['Country Name',train_year+1,'id']]
    choropleth_actual_df.rename(columns={train_year+1:'GDP Per Capita'},inplace=True)
    choropleth_actual_df = choropleth_actual_df[['Country Name','GDP Per Capita','id']]

    choropleth_df = choropleth_df[['Country Name','GDP Per Capita','id']]
    return choropleth_df, choropleth_actual_df

Choropleth Process for Mapping GDP Per Capita Results from Model onto the Map

In [82]:

def chart_compilation(choropleth_df, choropleth_actual_df, train_year, model, model_score):
    gdpactual= alt.Chart(
        world
          , title = ["Actual GDP Per Capita"
                    ,'for Year: ' + str(train_year +1)]).mark_geoshape(stroke='black' #'white'
                                                                ,fillOpacity=1,strokeWidth=.05).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=choropleth_actual_df, key = 'id', fields=['id','Country Name','GDP Per Capita'])
    ).encode(
    # tooltip='properties.geounit:N',
    color=alt.Color('GDP Per Capita:Q'
                    ,scale= alt.Scale(domain=[
                                                # 0,100000
                                            choropleth_actual_df['GDP Per Capita'].min()
                                              ,choropleth_actual_df['GDP Per Capita'].max()
                                              ]
                                        # ,range=['red','yellow','green']
                                        ,range=['lightgreen','darkblue']
                                      ))
    , tooltip=['Country Name:N', 'GDP Per Capita:Q']

    ).project(
    type='mercator'
    ).properties(
    width=800,
    height=800
    )
    gdpactual#.show()

    gdpforecast= alt.Chart(
        world
          , title = ["GDP Per Capita Forecasting : "
                    ,'Forecast Year: ' + str(train_year +1) 
                    ,'Model: '+model.__class__.__name__+ ' Score: '+str(round(model_score,4))]
                    ).mark_geoshape(stroke='black' #'white'
                                    ,fillOpacity=1,strokeWidth=.05).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=choropleth_df, key = 'id', fields=['id','Country Name','GDP Per Capita'])
    ).encode(
    # tooltip='properties.geounit:N',
    color=alt.Color('GDP Per Capita:Q'
                    ,scale= alt.Scale(domain=[
                                           # 0
                                        choropleth_df['GDP Per Capita'].min()
                                              ,choropleth_df['GDP Per Capita'].max()]
                                        # ,range=['red','yellow','green']
                                        ,range=['lightgreen','darkblue']
                                      ))
    , tooltip=['Country Name:N', 'GDP Per Capita:Q']

    ).project(
    type='mercator'
    ).properties(
    width=800,
    height=800
    )
    gdpforecast#.show()
    return alt.hconcat(gdpactual, gdpforecast)

In [86]:
train_year = 2020
filedate = '2023-04-01'

world = alt.topo_feature(data.world_110m.url, feature='countries')

for alpha_denom in range(1,5):
    print('alpha: ', alpha_denom, alpha_denom/4)
    percent_threshold = 0.009
    # percent_threshold = 0.02
    # alpha = alpha_denom/4
    alpha = .000000000000001
    # alpha = .9
    model, train_year, test_y, predictions, model_score, max_features, max_depth, ccp_alpha, dt_init_regr_score = dt_w_sig_cols(train_year, filedate, 'log2', 100, alpha,percent_threshold, alpha_denom)

    choropleth_df, choropleth_actual_df = choropleth_dfs(test_y, train_year, predictions)
    chart_compilation(choropleth_df, choropleth_actual_df, train_year, model, model_score).show()


alpha:  1 0.25
training year:  2020  date:  2023-04-01
                    DA_EUR_FS_ODX_D  DA_EUR_FS_ODX_I  DA_EUR_FS_ODX_ID  \
Country Name                                                             
Afghanistan                     0.0              0.0               0.0   
Albania                         0.0              0.0               0.0   
Algeria                         0.0              0.0               0.0   
Andorra                         0.0              0.0               0.0   
Angola                          0.0              0.0               0.0   
...                             ...              ...               ...   
Vietnam                         0.0              0.0               0.0   
West Bank and Gaza              0.0              0.0               0.0   
Yemen                           0.0              0.0               0.0   
Zambia                          0.0              0.0               0.0   
Zimbabwe                        0.0              0.0     

/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

Displaying chart at http://localhost:58245/
alpha:  2 0.5
training year:  2020  date:  2023-04-01
                    DA_EUR_FS_ODX_D  DA_EUR_FS_ODX_I  DA_EUR_FS_ODX_ID  \
Country Name                                                             
Afghanistan                     0.0              0.0               0.0   
Albania                         0.0              0.0               0.0   
Algeria                         0.0              0.0               0.0   
Andorra                         0.0              0.0               0.0   
Angola                          0.0              0.0               0.0   
...                             ...              ...               ...   
Vietnam                         0.0              0.0               0.0   
West Bank and Gaza              0.0              0.0               0.0   
Yemen                           0.0              0.0               0.0   
Zambia                          0.0              0.0               0.0   
Zimbabwe      

second data model for significant columns


/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

Displaying chart at http://localhost:58245/
alpha:  3 0.75
training year:  2020  date:  2023-04-01
                    DA_EUR_FS_ODX_D  DA_EUR_FS_ODX_I  DA_EUR_FS_ODX_ID  \
Country Name                                                             
Afghanistan                     0.0              0.0               0.0   
Albania                         0.0              0.0               0.0   
Algeria                         0.0              0.0               0.0   
Andorra                         0.0              0.0               0.0   
Angola                          0.0              0.0               0.0   
...                             ...              ...               ...   
Vietnam                         0.0              0.0               0.0   
West Bank and Gaza              0.0              0.0               0.0   
Yemen                           0.0              0.0               0.0   
Zambia                          0.0              0.0               0.0   
Zimbabwe     

/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

Displaying chart at http://localhost:58245/
alpha:  4 1.0
training year:  2020  date:  2023-04-01
                    DA_EUR_FS_ODX_D  DA_EUR_FS_ODX_I  DA_EUR_FS_ODX_ID  \
Country Name                                                             
Afghanistan                     0.0              0.0               0.0   
Albania                         0.0              0.0               0.0   
Algeria                         0.0              0.0               0.0   
Andorra                         0.0              0.0               0.0   
Angola                          0.0              0.0               0.0   
...                             ...              ...               ...   
Vietnam                         0.0              0.0               0.0   
West Bank and Gaza              0.0              0.0               0.0   
Yemen                           0.0              0.0               0.0   
Zambia                          0.0              0.0               0.0   
Zimbabwe      

/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_col_results = dt_col_results.append(iter_df)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/1855982633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

Displaying chart at http://localhost:58245/


In [87]:
chart_iterations_dict = {}


for iter in range(1,5):
    iter_df = dt_col_results[dt_col_results['iteration'] == iter].sort_values(by=['column_significance'], ascending=False).iloc[:10,:]

    print(iter_df)
    iter_df['count'] = iter_df['column']
    iter_df['count'] = [dt_col_results.groupby(['column']).size().loc[x] for x in iter_df['column']]
    print(iter_df.columns)


    chart_iterations_dict[iter] = alt.Chart(iter_df).mark_bar().encode(
    x = alt.X('column_significance:Q',scale = alt.Scale(domain=[dt_col_results['column_significance'].min(),
                                                                dt_col_results['column_significance'].max()])),  
    y="column:O"
    , tooltip='column_descr:O',

    color=alt.Color('count:Q', scale= alt.Scale(range=['lightgreen','darkblue'] ))

    ).properties(height=100, width = 700)

    print(iter_df)
    print(iter_df.columns)


choropleth_df, choropleth_actual_df = choropleth_dfs(test_y, train_year, predictions)
chart_compilation(choropleth_df, choropleth_actual_df, train_year, model, model_score).show()

alt.vconcat(chart_iterations_dict[1], chart_iterations_dict[2], chart_iterations_dict[3], chart_iterations_dict[4]).show()


  filename train_year first_model_score second_model_score max_features  \
0      NaN       2020          0.952356           0.952019         log2   
0      NaN       2020          0.952356           0.952019         log2   
0      NaN       2020          0.958866           0.948016         log2   
0      NaN       2020          0.952356           0.952019         log2   
0      NaN       2020          0.958866           0.948016         log2   
0      NaN       2020          0.958866           0.948016         log2   
0      NaN       2020          0.958866           0.948016         log2   
0      NaN       2020          0.952356           0.952019         log2   
0      NaN       2020          0.958866           0.948016         log2   
0      NaN       2020          0.958866           0.948016         log2   

  max_depth ccp_alpha percent_threshold             column  \
0       100       0.0             0.009  DBF_NUM_FS_ODX_IP   
0       100       0.0             0.009  GGCBP_G01

/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_19996/2099723327.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choropleth_actual_df.rename(columns={train_year+1:'GDP Per Capita'},inplace=True)


Displaying chart at http://localhost:58245/
Displaying chart at http://localhost:58245/
